# Lost in the fourth dimension

In [1]:
#: Our usual imports
import numpy as np  # the Python array package
import matplotlib.pyplot as plt  # the Python plotting package

We are going to load a four-dimensional (X, Y, Z, t) BOLD image called
`ds107_sub012_t1r2.nii`.   First, fetch the image from the web.

In [2]:
import nipraxis
image_fname = nipraxis.fetch_file('ds107_sub012_t1r2.nii')
# Show the filename of the downloaded file.
image_fname

Import the `nibabel` module, and load the image with nibabel to create an
image object.

In [3]:
#- Import nibabel
import nibabel as nib

#- Load the image
img = nib.load(image_fname)

# Show the result
img

In [4]:
assert 'img' in dir()
assert img is not ...
assert hasattr(img, 'header')

In the usual way get the floating point array data from this image and show
the image shape.

In [5]:
#- Get image array data from image object
data = img.get_fdata()

# Show the shape.
data.shape

Select the volume at time position 9 from 4D image data array, by slicing over
the last dimension.  Note the shape.

In [6]:
#- Get the 10th volume.
tenth_vol = data[..., 9]
# Show the shape.
tenth_vol.shape

Get the standard deviation across all voxels in this 3D volume:

In [7]:
#- Standard deviation across all voxels for 10th volume
ten_std = np.std(tenth_vol)
# Show the result
ten_std

In [8]:
assert np.allclose(ten_std, 387.1928)

Loop over all volumes in the 4D image and store the standard deviation for
each volume in a list. Plot these to see if there are any volumes with
particularly unusual standard deviation.

In [9]:
#- Get standard deviation for each volume; then plot the values
stds = []
for i in range(data.shape[-1]):
    vol = data[..., i]
    stds.append(np.std(vol))

# Show the result
plt.plot(stds)

The [SPM imaging analysis software](https://www.fil.ion.ucl.ac.uk/spm) uses a
measure called “global signal” to give a very rough estimate of the average
signal value within the brain. The idea is that you need a threshold to select
voxels with signal high enough to be inside the brain, and not in the air
around the brain, which has very low (near 0) signal.

The algorithm goes like this.

* Get a single 3D volume V;
* Calculate the mean signal of the voxels in V; call that M;
* Make a threshold T where T = M / 8.
* Select all voxel values in V that are greater than T; call these values W;
* Return the mean of all values in W.

See [SPM global image
signal](http://imaging.mrc-cbu.cam.ac.uk/imaging/PrinciplesStatistics#Global_image_signal).

In the SPM code, the algorithm is implemented in a MATLAB function called
`spm_global`.

I used the MATLAB script [get_global_signals.m](./get_global_signals.m)
to run the `spm_global` MATLAB function on the volumes of
`ds107_sub012_t1r2.nii`. The script saved the SPM global values to a text
file [global_signals.txt](./global_signals.txt). The first four lines of
the `global_signals.txt` file look like this:

```
376.53
375.75
375.26
376.01
```

Read these global values calculated by SPM into a list, and plot the values.

In [10]:
#- Read global signal values calculated by SPM, and plot
global_signals = np.loadtxt('global_signals.txt')
plt.plot(global_signals)

Now implement the algorithm above to recalculate the SPM global signal
for the first volume (volume index zero).

**Hint**: you will likely need to index using a Boolean (mask) array.
Remember, the steps are:

* Get a single 3D volume `V`;
* Calculate the mean signal of the voxels in `V`; call that `M`;
* Make a threshold `T` where `T = M / 8`.
* Select all voxel values in `V` that are greater than `T`; call these
  values `W`; Get the mean of all values in `W`.

You should get the same value as SPM - the first value you read from
`global_signals.txt`.

In [11]:
#- Apply algorithm for SPM global calculation to first volume
#- Get volume data.
vol = data[..., 0]
#- Calculate threshold
T = np.mean(vol)/ 8
#- Select voxels greater than T
msk = vol > T
voxels_within = vol[msk]
#- Calculate mean
mean_over_T = np.mean(voxels_within)
# Show the result
mean_over_T

In [12]:
# Check against value calculated by SPM
assert round(mean_over_T, 2) == 376.53

Make a function called `spm_global` that accepts a 3D array as input, and
returns the global signal using the SPM algorithm. You will see there is a line
to call that function on the first volume to show that it is working.

If you don't remember how to make your own functions, the cell has a skeleton
for the function, but it does not yet work correctly - it always returns the
value 42.

In [13]:
# Make an `spm_global` function that accepts a 3D array as input,
# and returns the global mean for the volume according to the SPM
# algorithm
def spm_global(vol):
    # vol is a 3D array.
    T = np.mean(vol) / 8
    return np.mean(vol[vol > T])

# Show the result from the calling the function on the first volume.
print(spm_global(data[:, :, :, 0]))

Make a function called `get_spm_globals` that accepts an image filename as
an argument. The function will load the image, get the array data for the
image, use your new `spm_global` function calculate the global value for
each volume, and return these values as a list.

In [14]:
# Write a function `get_spm_globals` that returns the global values
# for each volume
def get_spm_globals(fname):
    #- Load the image given by "fname".
    img = nib.load(fname)
    #- Get the data
    data = img.get_fdata()
    #- Calculate the SPM global value for each volume.
    spm_vals = []
    for i in range(data.shape[-1]):
        vol = data[..., i]
        spm_vals.append(spm_global(vol))
    return spm_vals  # Return the result.

# Call the function to get the result.
all_globals = get_spm_globals(image_fname)
# Plot the result.
plt.plot(all_globals)

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** by using Kernel - Restart and Run All.
- **Save and Checkpoint** from the `File` menu, to keep a record of your
  solution.

You might now want to look at [the
solution](four_dimensions_solution.ipynb).